In [81]:
import pandas as pd

In [82]:
csv_path = '../data/Emergency IKEM 2017-2023.csv' 
df = pd.read_csv(csv_path, delimiter=';')

In [83]:
df.head()

,patient_ID,visit_ID,EntryDate,EntryTime,EndDate,EndTime,age,AdmissionType,AdmissionType_,AdmissionRZP,AdmissionRZP_,VisitReason,VisitReason_,VisitType,VisitType_,DgCode,DischargeTo_Agg,DischargeTo_unit_agg,file
0,1037,1,01.01.2017,10:47:00,01.01.2017,13:14:30,73.9,100,bez doporučení,NaN,? typ záchranky,3,palpitace,2,opakovaná návštěva,I48.3: Typický flutter síní,outpatient,NaN,MUSE_20231118_022101_39000.xml
1,1037,1,01.01.2017,10:47:00,01.01.2017,13:14:30,73.9,100,bez doporučení,NaN,? typ záchranky,3,palpitace,2,opakovaná návštěva,I48.3: Typický flutter síní,outpatient,NaN,MUSE_20231118_022102_42000.xml
2,840,2,01.01.2017,11:13:00,01.01.2017,13:34:00,66.5,100,bez doporučení,NaN,? typ záchranky,1,bolest na hrudi,2,opakovaná návštěva,"I47.1: Perzistentní síňová tachykardie, EKV na...",outpatient,NaN,MUSE_20231118_022111_51000.xml
3,840,2,01.01.2017,11:13:00,01.01.2017,13:34:00,66.5,100,bez doporučení,NaN,? typ záchranky,1,bolest na hrudi,2,opakovaná návštěva,"I47.1: Perzistentní síňová tachykardie, EKV na...",outpatient,NaN,MUSE_20231118_022110_48000.xml
4,1500,3,01.01.2017,11:27:00,01.01.2017,14:25:00,78.5,100,bez doporučení,NaN,? typ záchranky,3,palpitace,2,opakovaná návštěva,I48.0: Paroxyzmální fibrilace síní,outpatient,NaN,MUSE_20231118_022120_59000.xml


## Preprocessing ##

In [84]:
# Combine EntryDate and EntryTime to one datetime column
df['entry_datetime'] = pd.to_datetime(df['EntryDate'] + ' ' + df['EntryTime'], format='%d.%m.%Y %H:%M:%S')
end_datetime = pd.to_datetime(df['EndDate'] + ' ' + df['EndTime'], format='%d.%m.%Y %H:%M:%S')
df['time_spent'] = (end_datetime - df['entry_datetime']).dt.total_seconds() / (24.0 * 60 * 60)

# Drop the original date and time columns
df = df.drop(['EntryDate', 'EntryTime', 'EndDate', 'EndTime'], axis=1)

# Drop columns with majority of na values. 15928 and 16537 respectively
df = df.drop(['AdmissionRZP', 'DischargeTo_unit_agg'], axis=1)

# Drop rows where DgCode is na (11 entries)
df_clea = df.dropna(subset=['DgCode'])


# Specify the order of columns for sorting
sort_order = ['patient_ID', 'visit_ID', 'entry_datetime']

# Sort the DataFrame based on the specified columns
df = df.sort_values(by=sort_order)

# Ignore repeated measurements taken during the same visit
df = df.drop_duplicates(subset=['patient_ID', 'visit_ID'], keep='first')

df.head()

,patient_ID,visit_ID,age,AdmissionType,AdmissionType_,AdmissionRZP_,VisitReason,VisitReason_,VisitType,VisitType_,DgCode,DischargeTo_Agg,file,entry_datetime,time_spent
2230,1,1459,68.1,100,bez doporučení,? typ záchranky,3,palpitace,2,opakovaná návštěva,I48.3: Flutter síní paroxysmální při antiarytm...,outpatient,MUSE_20231122_211154_57000.xml,2017-04-10 16:26:00,0.093970
3466,1,2253,68.2,100,bez doporučení,? typ záchranky,3,palpitace,2,opakovaná návštěva,I47.1: Síňová makroreentry tachykardie s nepra...,outpatient,MUSE_20231122_221512_86000.xml,2017-06-03 16:08:00,0.140278
3959,1,2584,68.3,101,přes RZP/LSPP,Praha,3,palpitace,2,opakovaná návštěva,"I48.4: Parox. makroreentry síň. tachykardie, n...",outpatient,MUSE_20231122_222724_38000.xml,2017-06-28 06:34:00,0.169444
9887,1,6589,69.1,101,přes RZP/LSPP,Praha,2,dušnost,2,opakovaná návštěva,I47.1: Supraventrikulární tachykardie,cardiology clinic,MUSE_20231122_101126_09000.xml,2018-05-02 15:12:00,0.078472
19395,1,13267,70.9,101,přes RZP/LSPP,Praha,1,bolest na hrudi,2,opakovaná návštěva,I48.1: Přetrvávající (perzistentní) fibrilace ...,outpatient,MUSE_20231120_222227_88000.xml,2020-02-12 09:56:00,0.143056


In [85]:
df.isna().sum()

patient_ID          0
visit_ID            0
age                 0
AdmissionType       0
AdmissionType_      0
AdmissionRZP_       0
VisitReason         0
VisitReason_        0
VisitType           0
VisitType_          0
DgCode             11
DischargeTo_Agg     0
file                0
entry_datetime      0
time_spent          0
dtype: int64

In [70]:
df['time_spent'].max()

10957.089583333332

In [41]:
df[df['DischargeTo_Agg'] == 'exitus']

,patient_ID,visit_ID,EndDate,EndTime,age,AdmissionType,AdmissionType_,AdmissionRZP,AdmissionRZP_,VisitReason,VisitReason_,VisitType,VisitType_,DgCode,DischargeTo_Agg,DischargeTo_unit_agg,file,datetime
15425,19180,10487,12.04.2019,06:15:00,71.4,102,od ambulantního lékaře,NaN,? typ záchranky,1,bolest na hrudi,1,první návštěva,I46.1: Náhlá srdeční smrt,exitus,NaN,MUSE_20231121_110225_47000.xml,2019-04-12 05:16:00
